# Multi Train Gradient Update

## Importing Libraries

In [1]:
from typing import Dict
import threading

import gym
import numpy as np
import torch

from stable_baselines3 import PPO as ALGO
from stable_baselines3.common.evaluation import evaluate_policy

/home/arnab/.miniconda3/envs/pydrl/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [2]:
# Hyper-Parameters
NUM_CLIENT_MODELS = 2
NUM_TRAINING_STEPS = 10_0
NUM_ITERATIONS = 40
ENV_NAME = 'CartPole-v1'


## Init. ENV and Model

In [3]:
env = gym.make(ENV_NAME)
global_model = ALGO(
    "MlpPolicy",
    env
)

client_models = [ALGO("MlpPolicy", gym.make(ENV_NAME)) for i in range(NUM_CLIENT_MODELS)]

## Functions to Evaluate Model and Train Model within Thread

In [4]:
def evaluate(model, env, message = '', verbose = 0):
    fitnesses = []
    iterations = 2
    for i in range(iterations):
        fitness, _ = evaluate_policy(model, env)
        if verbose == 1:
            print(i, fitness, end=" ")
        fitnesses.append(fitness)

    mean_fitness = np.mean(sorted(fitnesses))
    print(f'Type {message} Mean reward: {mean_fitness}')

In [5]:
def train(model, timesteps):
    # print('Starting Training')
    model.learn(total_timesteps=timesteps)
    # print('Completed Training')

In [6]:
def multithread_eval(client_models):
    # Create Threads
    client_threads = [] 
    for ci in range(NUM_CLIENT_MODELS):
        thread = threading.Thread(target=evaluate, args=(client_models[ci], gym.make(ENV_NAME), f'Trained Model {ci}'))
        client_threads.append(thread)

    # Start Threads
    for thread in client_threads:
        thread.start()

    # Join Threads (wait until thread is completely executed)
    for thread in client_threads:
        thread.join()

## Initial Evaluation

In [7]:
for model in client_models:
    model.set_parameters(global_model.get_parameters())

evaluate(global_model, env)

multithread_eval(client_models)

/home/arnab/.miniconda3/envs/pydrl/lib/python3.7/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Type  Mean reward: 9.6
Type Trained Model 0 Mean reward: 9.350000000000001
Type Trained Model 1 Mean reward: 9.6


## Train for 1K Steps and Evaluate

In [8]:
# Create Threads
client_threads = [] 
for i in range(NUM_CLIENT_MODELS):
    thread = threading.Thread(target=train, args=(client_models[i], NUM_TRAINING_STEPS))
    client_threads.append(thread)


# Start Threads
for thread in client_threads:
    thread.start()

# Join Threads (wait until thread is completely executed)
for thread in client_threads:
    thread.join()

evaluate(global_model, env)

multithread_eval(client_models)

Type  Mean reward: 9.5
Type Trained Model 0 Mean reward: 76.55000000000001
Type Trained Model 1 Mean reward: 189.25


## Apply Gradient and Evaluate

In [9]:
global_dict = global_model.policy.state_dict()

# Accumulate Client Parameters / Weights
for k in global_dict.keys():
    global_dict[k] = torch.stack([client_models[i].policy.state_dict()[k].float() for i in range(len(client_models))], 0).mean(0)

# Load New Parameters to Global Model
global_model.policy.load_state_dict(global_dict)

# Load New Parameters to clients
for model in client_models:
    model.policy.load_state_dict(global_model.policy.state_dict())

evaluate(global_model, env)

Type  Mean reward: 126.75


In [10]:
# Evaluation Before Iterated Training
evaluate(global_model, env, "Global Initial Model")

for i in range(NUM_ITERATIONS):
    print('Train Iter: ', i)

    # Create Threads
    client_threads = [] 
    for ci in range(NUM_CLIENT_MODELS):
        thread = threading.Thread(target=train, args=(client_models[ci], NUM_TRAINING_STEPS))
        client_threads.append(thread)


    # Start Threads
    for thread in client_threads:
        thread.start()

    # Join Threads (wait until thread is completely executed)
    for thread in client_threads:
        thread.join()

    # Evaluation after Training
    multithread_eval(client_models)

    # Accumulate Client Parameters / Weights
    global_dict = global_model.policy.state_dict()
    for k in global_dict.keys():
        global_dict[k] = torch.stack([client_models[i].policy.state_dict()[k].float() for i in range(len(client_models))], 0).mean(0)

    # Load New Parameters to Global Model
    global_model.policy.load_state_dict(global_dict)

    # Load New Parameters to clients
    for model in client_models:
        model.policy.load_state_dict(global_model.policy.state_dict())

    # Evaluate Updated Global Model
    evaluate(model, env, 'Global Updated Model', verbose=0)

Type Global Initial Model Mean reward: 146.35000000000002
Train Iter:  0
Type Trained Model 1 Mean reward: 340.55
Type Trained Model 0 Mean reward: 426.95
Type Global Updated Model Mean reward: 431.75
Train Iter:  1
Type Trained Model 0 Mean reward: 259.79999999999995
Type Trained Model 1 Mean reward: 393.55
Type Global Updated Model Mean reward: 404.04999999999995
Train Iter:  2
Type Trained Model 0 Mean reward: 353.8
Type Trained Model 1 Mean reward: 417.3
Type Global Updated Model Mean reward: 420.70000000000005
Train Iter:  3
Type Trained Model 1 Mean reward: 420.2
Type Trained Model 0 Mean reward: 433.0
Type Global Updated Model Mean reward: 437.95
Train Iter:  4
Type Trained Model 1 Mean reward: 447.29999999999995
Type Trained Model 0 Mean reward: 453.55
Type Global Updated Model Mean reward: 445.54999999999995
Train Iter:  5
Type Trained Model 0 Mean reward: 437.85
Type Trained Model 1 Mean reward: 488.2
Type Global Updated Model Mean reward: 476.2
Train Iter:  6
Type Trained Mo

In [ ]:
global_model.get_parameters()

In [ ]:
global_model.save('a2c_lunar_multiproc')

In [ ]:
# Exporting Params as JSON
## Function to Convert Params Dict to Flattened List
def flatten_list(params):
    """
    :param params: (dict)
    :return: (np.ndarray)
    """
    params_ = {}
    for key in params.keys():
        params_[key] = params[key].tolist()
    return params_
## Write Parameters to JSON File
import json

all_params = global_model.get_parameters()
pol_params = flatten_list(all_params['policy'])

all_params['policy'] = pol_params

with open('a2c_lunar_multiproc.json', 'w') as f:
    json.dump(all_params, f, indent='\t')

In [ ]:
# model_loaded = ALGO(
#     "MlpPolicy",
#     env
# )

# evaluate(model_loaded,env, verbose=1)

# import json
# with open('a2c_lunar_multiproc.json', 'w') as f:
#     new_params = json.load(f)

# loaded_pol_params = new_params['policy']
# for key in loaded_pol_params.keys():
#     loaded_pol_params[key] = th.tensor(loaded_pol_params[key])

# new_params['policy'] = loaded_pol_params

# model_loaded.set_parameters(new_params)

model_loaded = ALGO.load('a2c_lunar_multiproc', env)

In [ ]:
env.reset()
evaluate(model_loaded,env, verbose=1)